In [1]:
import os
import yaml
import sys
from collections import deque
from copy import deepcopy
from typing import(
    Dict,
    List,
    Optional,
    Tuple,
    Union
)

In [2]:
mod_path = os.path.abspath(os.path.join(os.getcwd(),'..','..'))
mod_path

'/Users/adebayobraimah/Desktop/projects/convert_source'

In [3]:
sys.path.append(mod_path)

In [4]:
from convert_source.cs_utils.const import (
    DEFAULT_CONFIG,
    BIDS_PARAM
)

In [5]:
from convert_source.cs_utils.utils import (
    list_in_substr
)

In [6]:
config = "config.default.yml"; config = os.path.abspath(config); config

'/Users/adebayobraimah/Desktop/projects/convert_source/config/config.test/config.default.yml'

In [41]:
def read_config(config_file: Optional[str] = "", 
                verbose: Optional[bool] = False
                ) -> Tuple[Dict[str,str],Dict,Dict,Dict,List[str]]:
    '''Reads configuration file and creates a dictionary of search terms for 
    each modality provided that each BIDS modality is used as a key via the 
    keyword 'modality_search'. Should BIDS related parameter descriptions need 
    to be used when renaming files, the related search and mapping terms can included 
    via the keywords 'bids_search' and 'bids_map', respectively. If these keywords 
    are not specified, then empty dictionaries are returned. Should exclusions be provided 
    (via the key 'exclude') then an exclusion list is created. Should this not be provided, 
    then an empty list is returned.

    BIDS modalities:
        - anat:
            - T1w, T2w, FLAIR, etc.
        - func:
            - bold
                - task:
                    - resting state, <task-name>
        - dwi
        - fmap

    Usage example:
        >>> [search_dict, bids_search, bids_map, meta_dict, exclusion_list] = read_config(config_file)
    
    Arguments:
        config_file: File path to yaml configuration file. If no file is used, then the default configuration file is used.
        verbose: Prints additional information to screen.
    
    Returns: 
        Tuple of dictionaries and a list that consists of:
            * search_dict: Nested dictionary of heuristic modality search terms for BIDS modalities.
            * bids_search: Nested dictionary of heuristic BIDS search terms.
            * bids_map: Corresponding nested dictionary of BIDS mapping terms to rename files to.
            * meta_dict: Nested dictionary of metadata terms to write to JSON file(s).
            * exclusion_list: List of exclusion terms.
    
    Raises:
        ConfigFileReadError: Error that arises if no heuristic search terms are provided.
    '''
    class ConfigFileReadError(Exception):
        pass

    if config_file:
        config_file: str = os.path.abspath(config_file)
    else:
        config_file: str = DEFAULT_CONFIG

    with open(config_file) as file:
        data_map: Dict[str,str] = yaml.safe_load(file)
        if verbose:
            print("Initialized parameters from configuration file")
    
    # Required modality search terms
    if any("modality_search" in data_map for element in data_map):
        if verbose:
            print("Categorizing search terms")
        search_dict: Dict[str,str] = data_map["modality_search"]
        del data_map["modality_search"]
    else:
        if verbose:
            print("Heuristic search terms required. Exiting...")
        raise ConfigFileReadError("Heuristic search terms required. Exiting...")
    
    # BIDS search terms
    if any("bids_search" in data_map for element in data_map):
        if verbose:
            print("Including BIDS related search term settings")
        bids_search: Dict[str,str] = data_map["bids_search"]
        del data_map["bids_search"]
    else:
        if verbose:
            print("No BIDS related search term settings")
        meta_dict: Dict = dict()
    
    # BIDS mapping terms
    if any("bids_map" in data_map for element in data_map):
        if verbose:
            print("Corresponding BIDS mapping settings")
        bids_map: Dict[str,str] = data_map["bids_map"]
        del data_map["bids_map"]
    else:
        if verbose:
            print("No BIDS mapping settings")
        meta_dict: Dict = dict()
    
    # Metadata terms
    if any("metadata" in data_map for element in data_map):
        if verbose:
            print("Including additional settings for metadata")
        meta_dict: Dict[str,Union[str,int]] = data_map["metadata"]
        del data_map["metadata"]
    else:
        if verbose:
            print("No metadata settings")
        meta_dict: Dict = dict()
    
    # Exclusion terms  
    if any("exclude" in data_map for element in data_map):
        if verbose:
            print("Exclusion option implemented")
        exclusion_list: List[str] = data_map["exclude"]
        del data_map["exclude"]
    else:
        if verbose:
            print("Exclusion option not implemented")
        exclusion_list: List = list()
        
    return (search_dict,
            bids_search,
            bids_map,
            meta_dict,
            exclusion_list)

In [8]:
def comp_dict(d1: Dict, 
              d2: Dict, 
              path: Optional[str] = "", 
              verbose: bool = False
             ) -> Union[bool,None]:
    '''Compares 2 dictionaries to see if they have matching keys, and that each key maps
    to a value (that is NOT of type None). This is performed recursively.
    
    Usage example:
        >>> comp_dict(d1, d2)
        True
        
    Arguments:
        d1: Input dictionary.
        d2: Input dictionary.
        path: Visualized path that has been traversed.
        verbose: Print verbose output.
        
    Returns:
        Boolean 'True', raises exceptions otherwise.
        
    Raises:
        KeyError: Error that arises if input dictionaries do not have matching keys.
        ValueError: Error that arises if one or more of the keys in either dictionary map
            NoneType values.
    '''
    for k in d1:
        if (k not in d2):
            if verbose:
                print (path, ":")
                print (k + " as key not in d2", "\n")
            raise KeyError("Input dictionaries do not have matching keys")
        else:
            if type(d1[k]) is dict:
                if path == "":
                    path = k
                else:
                    if verbose:
                        path = path + "->" + k
                    pass
                comp_dict(d1[k],d2[k], path, verbose)
            else:
                if d1[k] != d2[k] and (d1[k] is None or d2[k] is None):
                    if verbose:
                        print (path, ":")
                        print (" - ", k," : ", d1[k])
                        print (" + ", k," : ", d2[k])
                    raise ValueError("One or both input BIDS dictionaries map to NoneType values.")
    return True

In [9]:
def depth(d: Dict) -> int:
    '''Uses breadth-first search approach to find the depth of a dictionary.
    
    Usage example:
        >>> depth(d)
        3
    
    Arguments:
        d: Input dictionary.
        
    Returns:
        Number of levels in dictionary
    '''
    queue = deque([(id(d), d, 1)])
    memo = set()
    while queue:
        id_, o, level = queue.popleft()
        if id_ in memo:
            continue
        memo.add(id_)
        if isinstance(o, dict):
            queue += ((id(v), v, level + 1) for v in o.values())
    return level

In [10]:
def list_dict(d: Dict[str,str]
             ) -> List[Dict[str,str]]:
    '''Creates a list of dictionaries provided a nested dictionary. 
    
    Usage example:
        >>> list_dict(d)
        [d1, d2, ..., dn]
    
    Arguments:
        d: Input (nested) dictionary.
        
    Returns:
        List of dictionaries.
    '''
    arr: List = []
    for k,v in d.items():
        tmp: Dict = {k:v}
        arr.append(tmp)
    return arr

In [36]:
def proc_batch(s:str,
               search_dict: Dict,
               bids_search: Optional[Dict] = None,
               bids_map: Optional[Dict] = None
              ):
    '''
    * Work in progress *
    '''
    # use this function to iterate through image data files and subjects
    search_arr: List[str] = list_dict(d=search_dict)
    
    for i in search_arr:
        for k,v in i.items():
            if depth(i) == 3:
                for k2,v2 in v.items():
                    modality_type = k
                    modality_label = k2
                    mod_search = v2
                    print(f"{modality_type} - {modality_label} - {mod_search}")
                    # Do stuff here
                    # Search str with mod_search list of substrings
                    print(list_in_substr(in_list=mod_search,in_str=s))
                    if list_in_substr(in_list=mod_search,in_str=s):
                        bids_name_dict = search_bids(s=s,
                                                     bids_search=bids_search,
                                                     bids_map=bids_map,
                                                     modality_type=modality_type,
                                                     modality_label=modality_label)
            elif depth(i) == 4:
                for k2,v2 in v.items():
                    for k3,v3 in v2.items():
                        modality_type = k
                        modality_label = k2
                        task = k3
                        mod_search = v3
                        print(f"{modality_type} - {modality_label} - {task} - {mod_search}")
                        # Do stuff here
                        if list_in_substr(in_list=mod_search,in_str=s):
                            bids_name_dict = search_bids(s=s,
                                                         bids_search=bids_search,
                                                         bids_map=bids_map,
                                                         modality_type=modality_type,
                                                         modality_label=modality_label,
                                                         task=task)
    return bids_name_dict

In [12]:
search_dict,bids_search,bids_map,meta_dict,exclusion_list = read_config(config)

In [13]:
comp_dict(bids_search,bids_map,verbose=True)

True

In [31]:
# t = list_dict(search_dict)

In [15]:
# proc_batch(t)

anat - T1w - ['T1', 'T1w', 'TFE']
anat - T2w - ['T2', 'T2w', 'TSE']
func - bold - rest - ['rsfMR', 'rest', 'FFE', 'FEEPI']


In [37]:
proc_batch("T1_img.dcm",search_dict)

anat - T1w - ['T1', 'T1w', 'TFE']
True
anat - T2w - ['T2', 'T2w', 'TSE']
False
func - bold - rest - ['rsfMR', 'rest', 'FFE', 'FEEPI']


In [99]:
def search_bids(s: str,
                bids_search: Optional[Dict] = None,
                bids_map: Optional[Dict] = None,
                modality_type: Optional[str] = "",
                modality_label: Optional[str] = "",
                task: Optional[str] = ""):
    '''Performs search of BIDS (or related terms) provided there are bids_search, and bids_map dictionaries, and some input
    string (or file, represented as a string).
    
    Usage example:
        >>>
        
    Arguments:
        s: Input string (or file, represented as a string).
        bids_search: Heurestic BIDS related search terms.
        bids_map: Descriptive BIDS terms to be mapped to.
        modality_type: Modality type (e.g. 'anat', 'func', 'dwi' etc).
        modality_label: Modality label (e.g. 'T1w','bold','dwi' etc).
        task: Task label to ...
        
    Returns:
        Nested dictionary of BIDS descriptive naming related terms.
    '''
    # search string with BIDS search term
    # if term is present, add BIDS mapped name to BIDS name dictionary (global, constant)
    
    # iterate through

    bids_name_dict: Dict = deepcopy(BIDS_PARAM)
    
    if bids_search and bids_map:
        pass
    else:
        return None
    
    if modality_type and modality_label:
        pass
    else:
        return None
    
    if depth(bids_search[modality_type]) == 3:
        for (k1,v1),(k2,v2) in zip(bids_search[modality_type][modality_label].items(),bids_map[modality_type][modality_label].items()):
            for va,vb in zip(v1,v2):
                print(f"{k1} - {va} - {vb}")
                # print(list_in_substr(in_list=[va],in_str=s))
                if list_in_substr(in_list=[va],in_str=s):
                    bids_name_dict[modality_type]['modality_label'] = modality_label
                    bids_name_dict[modality_type][k1] = vb
    elif depth(bids_search[modality_type]) == 4:
        for (k1,v1),(k2,v2) in zip(bids_search[modality_type][modality_label].items(),bids_map[modality_type][modality_label].items()):
            for va,vb in zip(v1,v2):
                print(f"{k1} - {va} - {vb}")
                print(type(vb))
                # print(list_in_substr(in_list=[va],in_str=s))
                # if list_in_substr(in_list=[va],in_str=s):
                #    bids_name_dict[modality_type]['modality_label'] = modality_label
                #     bids_name_dict[modality_type][k1] = vb
    
#     return bids_name_dict

In [59]:
depth(bids_search['anat'])

3

In [67]:
# BIDS_PARAM

In [18]:
BIDS_PARAM['anat']

{'acq': '', 'ce': '', 'rec': '', 'run': '', 'modality_label': ''}

In [19]:
m1="anat"; m2="T1w"

In [20]:
bids_search[m1][m2]

{'acq': ['Axial'], 'rec': ['MoCo']}

In [84]:
search_bids("T1_img.dcm",bids_search,bids_map,'anat','T1w')

acq - Axial - 2D
rec - MoCo - MotionCorrected


In [85]:
bids_search['func']['bold']['rest']

{'acq': ['single'], 'ce': ['New_Contrast'], 'dir': None, 'rec': None}

In [100]:
search_bids("rest.dcm",bids_search,bids_map,'func','bold','rest')

rest - acq - acq
<class 'str'>
rest - ce - ce
<class 'str'>
rest - dir - dir
<class 'str'>
rest - rec - rec
<class 'str'>


In [48]:
b

{'info': {'sub': '', 'ses': ''},
 'anat': {'acq': '', 'ce': '', 'rec': '', 'run': '', 'modality_label': ''},
 'func': {'task': '',
  'acq': '',
  'ce': '',
  'dir': '',
  'rec': '',
  'run': '',
  'echo': '',
  'modality_label': ''},
 'dwi': {'acq': '', 'dir': '', 'run': '', 'modality_label': ''},
 'fmap': {'acq': '',
  'run': '',
  'case1': {'phasediff': '', 'magnitude1': '', 'magnitude2': ''},
  'case2': {'phase1': '', 'phase2': '', 'magnitude1': '', 'magnitude2': ''},
  'case3': {'magnitude': '', 'fieldmap': ''},
  'case4': {'ce': '', 'dir': '', 'modality_label': 'epi'}}}

In [40]:
c = deepcopy(b); c

{'info': {'sub': '', 'ses': ''},
 'anat': {'acq': '2D',
  'ce': '',
  'rec': 'MotionCorrected',
  'run': '',
  'modality_label': 'T1w'},
 'func': {'task': '',
  'acq': '',
  'ce': '',
  'dir': '',
  'rec': '',
  'run': '',
  'echo': '',
  'modality_label': ''},
 'dwi': {'acq': '', 'dir': '', 'run': '', 'modality_label': ''},
 'fmap': {'acq': '',
  'run': '',
  'case1': {'phasediff': '', 'magnitude1': '', 'magnitude2': ''},
  'case2': {'phase1': '', 'phase2': '', 'magnitude1': '', 'magnitude2': ''},
  'case3': {'magnitude': '', 'fieldmap': ''},
  'case4': {'ce': '', 'dir': '', 'modality_label': 'epi'}}}

In [9]:
from convert_source.cs_utils.const import (
    DEFAULT_CONFIG,
    BIDS_PARAM
)

In [11]:
BIDS_PARAM

{'info': {'sub': '', 'ses': ''},
 'anat': {'acq': '', 'ce': '', 'rec': '', 'run': '', 'modality_label': ''},
 'func': {'task': '',
  'acq': '',
  'ce': '',
  'dir': '',
  'rec': '',
  'run': '',
  'echo': '',
  'modality_label': ''},
 'dwi': {'acq': '', 'dir': '', 'run': '', 'modality_label': ''},
 'fmap': {'acq': '',
  'run': '',
  'case1': {'phasediff': '', 'magnitude1': '', 'magnitude2': ''},
  'case2': {'phase1': '', 'phase2': '', 'magnitude1': '', 'magnitude2': ''},
  'case3': {'magnitude': '', 'fieldmap': ''},
  'case4': {'ce': '', 'dir': '', 'modality_label': 'epi'}}}

In [12]:
from convert_source.cs_utils.img_dir import img_dir_list